# 다나와 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd

In [113]:
base_url = 'https://www.danawa.com/'
driver = webdriver.Chrome()
driver.get(base_url)
search_box = driver.find_element(By.XPATH, '//*[@id="AKCSearch"]')
search_box.send_keys('무선청소기')
search_box.send_keys(Keys.ENTER)
time.sleep(2)

In [114]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [115]:
prod_items = soup.select('li.prod_item')
print(len(prod_items))

43


In [116]:
# 상품명
name = prod_items[0].select('p.prod_name>a')[0].get_text().strip()
name

'베이스어스 차량용 청소기 A3 (해외구매)'

In [117]:
# 스펙 목록
spec = prod_items[0].select('.spec_list')[0].get_text().strip().replace('\n','').replace('\t','')
spec

'차량용청소기 / 무선 / 흡입력: 15,000Pa / 최대출력: 135W / 헤파필터 / 추가노즐: 틈새, 솔브러쉬 / LED라이트 / 사용시간: 45분 / 배터리 잔량표시 / 소음: 75dB / 충전방식: USB C타입 / 충전시간: 3~5시간 / 색상: 블랙, 실버 / 크기: 296x56.5mm'

In [118]:
# 가격
price = prod_items[0].select('li.rank_one>p.price_sect>a>strong')[0].text.strip().replace(',', '')
price

'47510'

In [119]:
# 1페이지에 대한 상품정보 추출
prod_data = []
for prod_item in prod_items:
    if 'ad_nvista' in prod_item['class']:
        continue
    try:
        name = prod_item.select('p.prod_name>a')[0].get_text().strip()
        spec = prod_item.select('.spec_list')[0].get_text().strip().replace('\n','').replace('\t','')
        price = prod_item.select('li.rank_one>p.price_sect>a>strong')[0].text.strip()
        prod_data.append([name, spec, price])
    except:
        pass
print(len(prod_data))

40


In [142]:
# 상품 정보 태그에서 원하는 정보를 추출하는 함수
def get_prod_items(prod_items):
    prod_data = []
    for prod_item in prod_items:
        if 'ad_nvista' in prod_item['class']:
            continue
        try:
            name = prod_item.select('p.prod_name>a')[0].get_text().strip()
            spec = prod_item.select('.spec_list')[0].get_text().strip().replace('\n','').replace('\t','')
            price = int(prod_item.select('li.rank_one>p.price_sect>a>strong')[0].text.strip().replace(',', ''))
            prod_data.append([name, spec, price])
        except:
            pass
    return prod_data

In [143]:
prod_data = get_prod_items(prod_items)
prod_data

[['LG전자 오브제컬렉션 코드제로 A9S AU9982GD + MO971GA',
  '핸디/스틱청소기+로봇청소기세트 / 무선형 / 흡입+물걸레겸용 / 흡입력: 220W / [구성] 와이드바닥 / 스팀물걸레 / 침구 / 솔형 / 틈새 / 올인원타워 / [배터리] 분리형 (2개) / 충전시간: 4시간 / 사용시간: 1시간 / [성능] 스마트인버터모터 / 헤파필터 / 5단계여과 / [기능] IoT기능 / 디스플레이표시창 / 자동물공급 / LED라이트 / 자동먼지비움 / [부가] 색상: 카밍그린 / 무게: 2.7kg / 크기(가로x세로x깊이): 300x1120x260mm / ※ 상품 옵션 확인 후 구매',
  1768690],
 ['LG전자 코드제로 A9S AT9300IA',
  '핸디스틱청소기 / 무선형 / 흡입전용 / 흡입력: 200W / [구성] 바닥 / 침구 / 솔형 / 틈새 / 올인원타워 / [배터리] 분리형 (1개) / 충전시간: 4시간 / 사용시간: 1시간 / [성능] 스마트인버터모터 / 분리형배기필터 / 헤파필터 / 5단계여과 / [기능] 물걸레(별도구매) / IoT기능 / 자동물공급 / 배터리잔량표시 / 자동먼지비움 / [부가] 색상: 아이언그레이+딥그레이 / 무게: 2.6kg / 크기(가로x세로x깊이): 250x1120x260mm',
  699390],
 ['보만 VC7210',
  '핸디스틱청소기 / 무선형 / 흡입전용 / [구성] 바닥 / 틈새 / 연장툴 / [배터리] 충전시간: 10시간 / 사용시간: 10~15분 / 헤파필터 / [부가] 색상: 블랙+오렌지 / 무게: 0.8kg / 크기(가로x세로x깊이): 145x440x115mm',
  49790],
 ['샤오미 미지아 K10 프로',
  '핸디스틱청소기 / 무선형 / 흡입전용 / 흡입력: 150AW / [구성] 바닥 / 물걸레 / 침구 / 솔형 / 틈새 / 연장툴 / 거치대 / [배터리] 분리형 (1개) / 충전시간: 4시간 / 사용시간: 1시간5분 / [성능] BLDC모터 / 헤파필터 / 

# 여러페이지에 걸친 다나와 검색 페이지 크롤링

In [152]:
base_url = 'https://www.danawa.com/'
driver = webdriver.Chrome()
driver.get(base_url)
search_box = driver.find_element(By.XPATH, '//*[@id="AKCSearch"]')
search_box.send_keys('무선청소기')
search_box.send_keys(Keys.ENTER)
time.sleep(2)

prod_data = []
cnt = 0

for i in range(1,11):
    cnt += 1
    next_p = driver.find_element(By.XPATH, f'//*[@id="paginationArea"]/div/div/a[{i}]').click()
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    prod_items = soup.select('li.prod_item')
    
    for prod_item in prod_items:
        if 'ad_nvista' in prod_item['class']:
            continue
        try:
            name = prod_item.select('p.prod_name>a')[0].get_text().strip()
            spec = prod_item.select('.spec_list')[0].get_text().strip().replace('\n','').replace('\t','')
            price = int(prod_item.select('li.rank_one>p.price_sect>a>strong')[0].text.strip().replace(',', ''))
            prod_data.append([name, spec, price])
        except:
            pass
    
    print(f'{cnt}페이지 끝')

1페이지 끝
2페이지 끝
3페이지 끝
4페이지 끝
5페이지 끝
6페이지 끝
7페이지 끝
8페이지 끝
9페이지 끝
10페이지 끝


In [153]:
len(prod_data)

400

In [146]:
def get_search_page_url(keyword, page):
    return 'https://search.danawa.com/dsearch.php?query={}&volumeType=allvs&page={}&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&mode=simple&recommendedSort=Y&defaultUICategoryCode=14237761&defaultPhysicsCategoryCode=1840%7C59152%7C59189%7C0&defaultVmTab=3014&defaultVaTab=969214&tab=main'.format(keyword, page)
#     return f'https://search.danawa.com/dsearch.php?query={'검색어'}&volumeType=allvs&page={'페이지'}&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&mode=simple&recommendedSort=Y&defaultUICategoryCode=14237761&defaultPhysicsCategoryCode=1840%7C59152%7C59189%7C0&defaultVmTab=3014&defaultVaTab=969214&tab=main'

In [147]:
keyword = '무선청소기'
page = 10
url = get_search_page_url(keyword, page)
print(url)

https://search.danawa.com/dsearch.php?query=무선청소기&volumeType=allvs&page=10&limit=40&sort=saveDESC&list=list&boost=true&addDelivery=N&mode=simple&recommendedSort=Y&defaultUICategoryCode=14237761&defaultPhysicsCategoryCode=1840%7C59152%7C59189%7C0&defaultVmTab=3014&defaultVaTab=969214&tab=main


In [148]:
from tqdm.notebook import tqdm

In [149]:
# 전체 페이지 코드 정리
keyword = '무선청소기'
total_page = 10
prod_data_total = []

for page in tqdm(range(1, total_page+1)):
    url = get_search_page_url(keyword, page)
    driver.get(url)
    time.sleep(3)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 상품정보 추출
    prod_items = soup.select('li.prod_item')
    prod_item_list = get_prod_items(prod_items)
    
    # 추출한 정보 저장
    prod_data_total += prod_item_list
    

  0%|          | 0/10 [00:00<?, ?it/s]

In [150]:
df = pd.DataFrame(prod_data_total, columns=['상품명', '스펙', '가격'])
df

,상품명,스펙,가격
0,베이스어스 차량용 청소기 A3 (해외구매),"차량용청소기 / 무선 / 흡입력: 15,000Pa / 최대출력: 135W / 헤파필...",47510
1,LG전자 오브제컬렉션 코드제로 A9S AU9982,핸디스틱청소기 / 무선형 / 흡입+물걸레겸용 / 흡입력: 220W / [구성] 와이...,1019160
2,샤오미 미지아 차량용 핸디 무선 청소기 (해외구매),"차량용청소기 / 무선 / 흡입력: 13,000Pa / 최대출력: 120W / 헤파필...",38990
3,LG전자 오브제컬렉션 코드제로 A9S AU9882,핸디스틱청소기 / 무선형 / 흡입+물걸레겸용 / 흡입력: 220W / [구성] 와이...,894400
4,삼성전자 비스포크 제트 VS20A956A3,핸디스틱청소기 / 무선형 / 흡입전용 / 흡입력: 210W / [구성] 바닥 / 솔...,492190
...,...,...,...
395,유피 무선 차량 청소기 H20,"차량용청소기 / 무선 / 흡입력: 14,000Pa / 추가노즐: 틈새, 솔브러쉬 /...",125390
396,삼성전자 비스포크 제트 VS20A957E2,핸디스틱청소기 / 무선형 / 흡입+물걸레겸용 / 흡입력: 210W / [구성] 바닥...,708990
397,삼성전자 제트 VS20T9253QD,핸디스틱청소기 / 무선형 / 흡입전용 / 흡입력: 200W / [구성] 바닥 / 솔...,417600
398,샤오미 미홀 H17 Pro,물걸레청소기 / 롤러식 / 물걸레+진공 / 무선형 / [배터리] 리튬이온 (분리형)...,215100


In [151]:
df.to_csv("./csv/danawa_crawling.csv", index=False)